In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")


In [43]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = 'gemini-1.5-pro',
    temperature= .2,
    api_key= api_key,
    max_retries= 2,
    max_tokens= 1000
)


response = llm.invoke("Who is sachin tendulkar?")

print(response.content)

Sachin Tendulkar is widely regarded as one of the greatest cricketers of all time.  He is an Indian former international cricketer who played for India for 24 years.  He holds numerous batting records, including the most runs in both Test and One Day International (ODI) cricket, as well as the most centuries in both formats.  He is also the only player to have scored one hundred international centuries, the first batsman to score a double century in a One Day International, and the only player to complete more than 30,000 runs in international cricket.  He is often nicknamed "The God of Cricket" by his fans.  He retired from all forms of cricket in November 2013.


In [21]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("Source\Budget_Circular_2024_25.pdf")

document = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function=len
)

chunks = splitter.split_documents(document)

len(chunks)

<>:4: SyntaxWarning: invalid escape sequence '\B'
<>:4: SyntaxWarning: invalid escape sequence '\B'
C:\Users\DELL\AppData\Local\Temp\ipykernel_12128\2885317271.py:4: SyntaxWarning: invalid escape sequence '\B'
  loader = PyPDFLoader("Source\Budget_Circular_2024_25.pdf")


406

In [27]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    google_api_key= api_key,
    model= 'models/embedding-001'

)

In [31]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(chunks, embeddings)



In [33]:
query = "what is the networth of CPSE?"

docs = db.similarity_search_with_score(query)
    
docs

[(Document(id='74dbfe7d-0f37-4a5c-97c7-4c5e5e02b6c5', metadata={'source': 'Source\\Budget_Circular_2024_25.pdf', 'page': 85}, page_content="(iv) Long term borrowing and further capacity to borrow on the basis of its 'Net worth'; \n(v) Any other fmancial commitments in the near future; \n(vi) Business /other receivables and contingent liabilities , if any; and \n(vii) Market price/book value of share. \n6.4 Based on this analysis, it needs to be clearly brought out that surplus cash and bank \nbalance with the CPSE shall be considered for restructuring of capital through buyback. \nHowever, every CPSE having net-worth of atleast Rs. 2000 crore and cash and bank \nbalance of over Rs. 1000 crore shall exercise the option to buy-back their shares. \n7. Issue of Bonus Shares: \n7.1 The Department of Public Enterprises had issued guidelines on issue of bonus shares \nby Public Sector Undertakings vide O.M. No. DPE / 12(6) /95-Fin. Dated lOth November \n1995 and O.M. No. DPE / 13(21)-Fin. Dat

In [47]:
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

retriever = db.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

response = retrieval_chain.invoke({"input":query})

print(response['answer'])


c:\Users\DELL\anaconda3\envs\py_rag\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Net-worth, as referred to in the guidelines, has the same meaning as defined in the Companies Act, 2013, as amended from time to time.  A CPSE having net-worth of at least Rs. 2000 crore *and* cash and bank balance of over Rs. 1000 crore shall exercise the option to buy-back their shares.
